# ALBERT
[ALBERT](https://arxiv.org/pdf/1909.11942.pdf),A Lite Bert    
参考链接：   
https://amitness.com/2020/02/albert-visual-summary/

## BERT 的缺点
内存限制和通讯开销：
- BERT非常大的模型，巨量的训练参数，想要从头开始训练，需要大量的计算资源，且计算机内存会限制模型大小
- 一种解决方法就是分布式训练；数据被拆分到多个机器上，单独进行训练，然后再同步不同的梯度，大量参数的同步过程造成较大的通讯开销，延缓训练过程；同理 模型并行化 时，也会造成同样的瓶颈 

模型退化：
- 模型变大，并不一定会获得更好的性能，反而性能可能下降

## ALBERT的改进
 ALBERT 本质就是对 BERT 模型压缩后的产物。
 > 模型压缩有很多手段，包括剪枝，参数共享，低秩分解，网络结构设计，知识蒸馏等
 
 ALBERT降低了模型参数量从而降低了模型的训练时间（通信开销降低），但是，**模型的预测推理时间并没有降低**。

### Cross-layer parameter sharing
- BERT-LARGE 有 24 层，而 BERT-BASE 有 12 层；随着层数增加，参数量指数级增加   
      
      
- ALBERT 提出了 跨层参数共享 的概念，不是单独学习 BERT 每一层的参数，而是将每一层的参数设置为相同
    - 可以只共享 feed-forward 层的参数，或 attention 层的参数，或整个编码层的参数
    - 共享整层参数时，模型参数量明显下降，从 110M 下降到 31M 
    
- 参数共享对性能的影响很小，且能够帮助模型稳定网络的参数。这点是通过L2距离与 cosine similarity 得出的

### SOP 替代 NSP
- SOP 全称为 Sentence Order Prediction，其用来取代 NSP 在 BERT 中的作用，
    - ROBERTA 和 XLNET 模型显示 NSP 非但没有作用，反而会对模型带来一些损害。删除 NSP 任务，模型在一些列任务上的性能会提升
    - NSP 相比 MLM 并不是一个难的任务，NSP 设置到主题预测和连贯性预测，主题预测比较容易学习，因为与 MLM 任务重合；因此就算没有学到 连贯性预测，NSP 也会得到较高的精度
    
      
- SOP的方式与NSP相似，也是判断第二句话是不是第一句话的下一句，但对于负例来说，SOP并不从不相关的句子中生成，而是将原来连续的两句话翻转形成负例。

### Factorized embedding parameterization

通过对Embedding 部分降维来达到降低参数的作用。
- BERT-Base 中 Embedding 层的维度与隐层的维度一样都是768；  
- 而对于词的分布式表示，如 Word2Vec 就多采用50或300这样的维度。
- 通过将 Embedding 部分分解来达到降低参数量的作用，其以公式表示如下：

$$O(V\times H) \rightarrow O(V\times E+E\times H)$$

    V：词表大小；H：隐层维度；E：词向量维度

以 BERT-Base 为例，    
- 词表大小为3w，此时的参数量为：768 * 3w = 23040000。 
- 如果将 Embedding 的维度改为 128，那么此时Embedding层的参数量为： 128 * 3w + 128 * 768 = 3938304。
- Embedding参数量从原来的23M变为了现在的4M，
- 但从整个模型来看，BERT-Base的参数量在 110M，降低19M也不能产生什么革命性的变化。
- 可以说 Embedding 层的因式分解其实并不是降低参数量的主要手段。
- 意忽略了Position Embedding的那部分参数量， 主要是考虑到512相对于3W显得有点微不足道。

### ALBERT 的性能提升
- 参数量比 BERT-large 小 18 倍
- 训练比 BERT 快 1.7 倍
- 在 GLUE, RACE 和 SQUAD 任务上获得最佳性能